# Spatial Analysis and Modeling

### Instructor: Rossano Schifanella
Email: [rossano.schifanella@unito.it](mailto:rossano.schifanella@unito.it)

## Represent Spatial Data

## Exercise 2

In this exercise you will learn how to:

* Connect a spatial database to a python script
* Make simple spatial queries
* Join spatial tables and different geometry types in complex queries

## Connect to a PostGIS database

In [12]:
import psycopg2
import psycopg2.extras

try:
    conn = psycopg2.connect("dbname='geocycle' user='geocycle' host='localhost' password='geocycle'")
except Exception as e:
    print(e)

## Make simple spatial queries

**Q1: _Which is the extension (in km<sup>2</sup>) of the different boroughs in New York City?_**


In [13]:
cursor = conn.cursor()

cursor.execute("SELECT name, boroname, ST_Area(geom::geography)/power(10,6) AS area "\
               "FROM nyc_neighborhoods "\
               "ORDER by area DESC");

records = cursor.fetchall()

for r in records:
    print("%30s\t%20s\t%.2f" %(r[0], r[1], r[2]))

cursor.close()


                 The Rockaways	              Queens	27.97
      Gravesend-Sheepshead Bay	            Brooklyn	19.93
                      Canarsie	            Brooklyn	17.71
       Douglastown-Little Neck	              Queens	17.54
     Bloomfield-Chelsea-Travis	       Staten Island	14.13
                       Jamaica	              Queens	13.86
                  Richmondtown	       Staten Island	12.91
            Mapleton-Flatlands	            Brooklyn	12.20
                 East Brooklyn	            Brooklyn	11.10
                   Eastchester	           The Bronx	10.79
            Bedford-Stuyvesant	            Brooklyn	10.78
                        Utopia	              Queens	10.70
                      Flushing	              Queens	10.63
                  New Brighton	       Staten Island	9.28
                   Saintalbans	              Queens	9.08
                  Nkew Gardens	              Queens	8.65
                  Forest Hills	              Queens	8.57
                  

**Q2: _Which is the longest street in New York City?**

Note the use of the class **DictCursor** that allows to access to the retrieved records using an interface similar to the Python dictionaries instead of the tuples. 

In [14]:
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

cursor.execute("SELECT id as wayid, "\
        "ST_Length(linestring::geography) as length, "\
        "tags->'name' as name, "\
        "tags->'highway' as type "\
        "FROM ways "\
        "WHERE linestring IS NOT NULL ORDER BY length DESC LIMIT 1;");

records = cursor.fetchall()

for r in records:
    print("%8d\t%.3f\t%24s\t%s" %(r["wayid"], r["length"], r["name"], r["type"]))  
    
cursor.close()

 5703776	7651.028	               Boardwalk	pedestrian


**Q3: _How many census blocks in New York City have a hole in them?_** 

In [15]:
cursor = conn.cursor()

cursor.execute(
    "SELECT count(*) "\
        "FROM nyc_census_blocks "\
        "WHERE ST_NumInteriorRings(ST_GeometryN(geom, 1)) > 0;");

print(cursor.fetchone()[0])


37


**Q4: _What is the length (in kilometers) of streets in New York City, summarized by type?_** 

In [16]:
cursor = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

cursor.execute(
    "SELECT tags->'highway' as type, "\
        "SUM(ST_Length(linestring::geography))/1000 AS length "\
        "FROM ways "\
        "GROUP BY type "\
        "ORDER BY length DESC;");

records = cursor.fetchall()

for r in records:
    print("%15s\t%.2f" %(r["type"], r["length"]))


    residential	7383.14
      secondary	1062.90
        service	936.78
        footway	719.27
       motorway	664.02
       tertiary	641.90
        primary	540.17
  motorway_link	408.60
       cycleway	251.87
   unclassified	216.33
          trunk	187.04
     pedestrian	93.35
           path	69.59
          track	44.79
   primary_link	14.17
     trunk_link	13.77
          steps	12.76
      bridleway	11.73
 secondary_link	7.05
   construction	3.27
        raceway	2.61
  tertiary_link	2.31
       platform	1.36
  living_street	1.02
           road	0.71
       proposed	0.59
         closed	0.32
        disused	0.27
       elevator	0.01


**Q5: __** 



## Make spatial joins and complex queries

In this section we will show how to make spatial joins and the use of spatial functions that involve more than a geometry (e.g., ST_Within, ST_DWithin, ST_Contains, or ST_Crosses)

**Q1: _How many Instagram pictures has been taken in each borough during March 2015?_**

In [17]:

cursor.execute(
    "SELECT b.boroname as boroname, count(p.pid) AS cnt "\
    "FROM instagram_photos as p "\
    "JOIN nyc_boroughs as b "\
    "ON ST_Within(p.geom, b.geom) "\
    "WHERE date_part('month', p.created_at)=3 "\
    "GROUP BY b.boroname "\
    "ORDER BY cnt DESC;");

records = cursor.fetchall()

for r in records:
    print("%20s\t%d" %(r["boroname"], r["cnt"]))


           Manhattan	7636


**Q2: _Which were the 10 most dangerous streets of New York city in March?_**

We suppose that a criminal activity happended in a street within a 25 meters confidence buffer

In [18]:
cursor.execute(
    "SELECT w.id as wayid, w.tags->'name' AS wayname, w.tags->'highway' AS waytype, count(c.identifier) AS cnt "\
    "FROM (SELECT * FROM nyc_crime WHERE date_part('month', date)=3) AS c "\
    "JOIN ways AS w "\
    "ON ST_DWithin(c.geom, w.linestring, 0.0001) "\
    "GROUP BY w.id "\
    "ORDER BY cnt DESC "\
    "LIMIT 10;")
    
records = cursor.fetchall()

for r in records:
    print("%10s\t%20s\t%20s\t%d" %(r["wayid"], r["wayname"], r["waytype"], r['cnt']))


 195743336	          5th Avenue	           secondary	37
   5695685	                None	             service	32
   5697592	        Hazen Street	             service	32
 196117039	    Lexington Avenue	           secondary	27
   5670600	    West 34th Street	             primary	25
   5669742	            Broadway	           secondary	23
 196117089	          3rd Avenue	             primary	22
 221157083	          3rd Avenue	             primary	20
 221696378	     Nostrand Avenue	           secondary	17
 221576343	      Jamaica Avenue	           secondary	15


**Q3: Which is the neighborhood that has the highest number of streets that cross its border? **

In [19]:
cursor.execute(
    "select n.name as name, count(w.id) as cnt "\
    "from ways as w "\
    "join nyc_neighborhoods as n "\
    "on ST_Crosses(w.linestring, n.geom) "\
    "group by n.gid "\
    "order by cnt desc "\
    "limit 1;")

records = cursor.fetchall()

for r in records:
    print("%10s\t%d" %(r["name"], r["cnt"]))



  Bushwick	180


**Q4: _Which is the most frequent type of crime in each neighborhood and its relative frequency?_**

In [20]:
cursor.close()
conn.close()